In [ ]:
 import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# Load the cleaned dataset
df_cleaned = pd.read_csv('/kaggle/working/cleaned_supply_chain_data.csv')

# --- Distribution Fitting Prototype (Manual Implementation) ---

# Define key variables for distribution fitting
key_variables = {
    'Inter-Arrival Time (Days)': df_cleaned['inter_arrival_time'].dropna(),
    'Order Profit Per Order': df_cleaned['order_profit_per_order'].dropna()
}

# Define distributions to fit and their corresponding scipy.stats functions
distributions = {
    'Exponential': stats.expon,
    'Weibull': stats.weibull_min,
    'Lognormal': stats.lognorm,
    'Pareto': stats.pareto
}

# Store results for comparison table
fitting_results = []

# Function to calculate AIC and BIC
def calculate_aic_bic(data, dist_func, params):
    # Filter out data points that would result in log(0) or NaN for the PDF
    # For Pareto and Lognormal, data must be positive.
    # For inter-arrival time, we already filtered out 0s for positive distributions.
    # For profit, if negative values are present, some PDFs might return 0 or NaN.
    # We need to handle this to avoid errors in log-likelihood calculation.
    if dist_func == stats.lognorm or dist_func == stats.pareto:
        data_filtered = data[data > 0]
    else:
        data_filtered = data

    if data_filtered.empty:
        return np.nan, np.nan

    try:
        log_likelihood = np.sum(dist_func.logpdf(data_filtered, *params))
        k = len(params) # Number of parameters
        n = len(data_filtered) # Number of data points used
        aic = -2 * log_likelihood + 2 * k
        bic = -2 * log_likelihood + k * np.log(n)
        return aic, bic
    except Exception as e:
        # print(f"Error calculating AIC/BIC for {dist_func.name} with params {params}: {e}")
        return np.nan, np.nan

# Perform fitting for each key variable
for var_name, data_series in key_variables.items():
    print(f"\n--- Fitting Distributions for {var_name} ---")

    # For Pareto and Lognormal, data must be strictly positive.
    # For 'Inter-Arrival Time', 0 represents the first order, which is not an "arrival interval".
    # We will filter out 0 for these distributions.
    if var_name == 'Inter-Arrival Time (Days)':
        data_for_positive_dist = data_series[data_series > 0].copy()
    else:
        # For 'Order Profit Per Order', it contains negative values.
        # Exponential, Weibull, Lognormal, Pareto are typically for positive values.
        # Fitting these to profit data with negative values will cause errors or inappropriate fits.
        # For the purpose of demonstration and adhering to the prompt,
        # I will attempt to fit only on the positive profit values for Lognormal and Pareto.
        # For Exponential and Weibull, they also usually expect non-negative.
        # If the user intends to model profit/loss, a different family of distributions or transformation might be needed.
        # Here, let's explicitly note this limitation for profit.
        data_for_positive_dist = data_series.copy()


    for dist_name, dist_func in distributions.items():
        params = None
        ks_pvalue = np.nan
        aic = np.nan
        bic = np.nan

        # Special handling for distributions that require positive values
        current_data_for_fit = data_for_positive_dist
        if (dist_name in ['Lognormal', 'Pareto']) or \
           (dist_name in ['Exponential', 'Weibull'] and (data_for_positive_dist < 0).any()):
            current_data_for_fit = data_for_positive_dist[data_for_positive_dist > 0].copy()
            if current_data_for_fit.empty:
                print(f"  Skipping {dist_name} for {var_name} due to no positive data points.")
                continue

        try:
            # Fit the distribution
            params = dist_func.fit(current_data_for_fit)

            # Calculate KS test
            ks_stat, ks_pvalue = stats.kstest(current_data_for_fit, dist_func.name, args=params)

            # Calculate AIC and BIC
            aic, bic = calculate_aic_bic(current_data_for_fit, dist_func, params)

            print(f"  {dist_name} Fitted Parameters: {params}, KS p-value: {ks_pvalue:.4f}, AIC: {aic:.2f}, BIC: {bic:.2f}")

            fitting_results.append({
                'Variable': var_name,
                'Distribution': dist_name,
                'Parameters': params,
                'KS p-value': ks_pvalue,
                'AIC': aic,
                'BIC': bic
            })

            # Plotting the fitted PDF for visual inspection
            plt.figure(figsize=(10, 6))
            sns.histplot(current_data_for_fit, bins=50, kde=False, stat='density', color='gray', label='Empirical Data')
            # Generate points for the fitted PDF
            xmin, xmax = plt.xlim()
            x = np.linspace(xmin, xmax, 100)
            p = dist_func.pdf(x, *params)
            plt.plot(x, p, 'r-', lw=2, label=f'Fitted {dist_name}')
            plt.title(f'Distribution Fit for {var_name}: {dist_name}')
            plt.xlabel(var_name)
            plt.ylabel('Probability Density')
            plt.legend()
            plt.savefig(f'fit_plot_{var_name.replace(" ", "_").lower()}_{dist_name.lower()}.png')
            plt.close()

        except Exception as e:
            print(f"  Error fitting {dist_name} for {var_name}: {e}")
            fitting_results.append({
                'Variable': var_name,
                'Distribution': dist_name,
                'Parameters': 'Error',
                'KS p-value': np.nan,
                'AIC': np.nan,
                'BIC': np.nan
            })

# Create the comparison table DataFrame
comparison_df = pd.DataFrame(fitting_results)

# Sort by Variable and then by AIC (lower is better)
comparison_df.sort_values(by=['Variable', 'AIC'], inplace=True)

print("\n--- Distribution Fitting Comparison Table ---")
print(comparison_df.to_markdown(index=False))

# Optional: Save the comparison table to a CSV or Excel file
comparison_df.to_csv('distribution_fitting_comparison_table.csv', index=False)
print(f"\nDistribution fitting comparison table saved to distribution_fitting_comparison_table.csv")